In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np

label_columns = ['pornographic-content', 'violence', 'death', 'sexual-assault', 'abuse', 'blood',
                 'suicide', 'pregnancy', 'child-abuse', 'incest', 'underage', 'homophobia',
                 'self-harm', 'dying', 'kidnapping', 'mental-illness', 'dissection',
                 'eating-disorders', 'abduction', 'body-hatred', 'childbirth', 'racism',
                 'sexism', 'miscarriages', 'transphobia', 'abortion', 'fat-phobia',
                 'animal-death', 'ableism', 'classism', 'misogyny', 'animal-cruelty']

train = load_dataset("json", 
                     data_files="/kaggle/input/pan23trigger/pan23-trigger-detection/pan23-trigger-detection-train/works.jsonl", 
                     streaming=False, split="train")

val = load_dataset("json", 
                     data_files="/kaggle/input/pan23trigger/pan23-trigger-detection/pan23-trigger-detection-validation/works.jsonl", 
                     streaming=False, split="train")

def to_df(data):
    df = pd.DataFrame(data)

    # One-hot encode the "labels" column
    onehot_labels = pd.DataFrame(np.array(df['labels'].tolist()), columns=label_columns)

    # Remove the original "labels" column from the DataFrame
    df.drop(columns=['labels'], inplace=True)

    # Concatenate the one-hot encoded labels with the original DataFrame
    encoded_df = pd.concat([df, onehot_labels], axis=1)

    return encoded_df

train = to_df(train)
val = to_df(val)

train

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ddba235724ea2167/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-cb1306363117a2b2/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


,work_id,text,pornographic-content,violence,death,sexual-assault,abuse,blood,suicide,pregnancy,...,sexism,miscarriages,transphobia,abortion,fat-phobia,animal-death,ableism,classism,misogyny,animal-cruelty
0,23758279,"<div class=""userstuff""><p>\n <em>“A little le...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,23780866,"<div class=""userstuff""><hr><p align=""center"">&...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23754256,"<div class=""userstuff""><hr><p>&nbsp;</p><p>""Ba...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,23757385,"<div class=""userstuff""><p>It was near midnight...",0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,23771095,"<div class=""userstuff""><p class=""western"">&nbs...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307097,19181050,"<div class=""userstuff""><p>The first time Jin h...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307098,19179817,"<div class=""userstuff""><p>Ah, love. It was som...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307099,19197943,"<div class=""userstuff""><p>Clarke wakes up earl...",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
307100,19164955,"<div class=""userstuff""><p>Sergio felt tense wh...",1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def undersample_class_combinations(df, label_columns):
    # Get the class combinations for each data entry
    class_combinations = df[label_columns].apply(tuple, axis=1)
    combination_counts = Counter(class_combinations)
    
    # Print the top 15 most common classes and their counts before undersampling
    most_common_classes_before = combination_counts.most_common(15)
    print("Top 15 Most Common Classes Before Undersampling:")
    for combination, count in most_common_classes_before:
        print(f"Class Combination: {combination}, Count: {count}")

    # Plot class combinations before undersampling
    plt.figure(figsize=(10, 12))
    plt.subplot(2, 1, 1)
    plt.bar(range(len(combination_counts)), list(combination_counts.values()))
    plt.title('Class Combinations Before Undersampling')
    plt.ylabel('Count (log scale)')
    plt.yscale('log') #symlog
    plt.xticks([])

    # Sort class combinations by count in descending order
    sorted_combinations = sorted(combination_counts.items(), key=lambda x: x[1], reverse=True)

    # Calculate the maximum allowed count for the majority class combinations
    max_count = sorted_combinations[0][1]
    second_max_count = sorted_combinations[1][1]
    max_count_limit = int(second_max_count * 5)

    # Create a dictionary to map class combinations to indices
    combination_indices = {}
    for i, cc in enumerate(class_combinations):
        if cc not in combination_indices:
            combination_indices[cc] = [i]
        else:
            combination_indices[cc].append(i)

    # Undersample the majority class combinations
    undersampled_indices = [random.sample(indices, max_count_limit) for _, indices in tqdm(combination_indices.items()) if len(indices) > max_count_limit]
    undersampled_indices = [idx for indices in undersampled_indices for idx in indices]

    # Create the balanced DataFrame
    df_balanced = df.iloc[undersampled_indices].copy()
    df_balanced.reset_index(drop=True, inplace=True)

    # Include entries from other classes
    for _, indices in tqdm(combination_indices.items()):
        if len(indices) <= max_count_limit:
            df_balanced = pd.concat([df_balanced, df.iloc[indices]])

    # Update the counts of class combinations after undersampling
    class_combinations_balanced = df_balanced[label_columns].apply(tuple, axis=1)
    combination_counts_balanced = Counter(class_combinations_balanced)

    # Plot class combinations after undersampling
    plt.subplot(2, 1, 2)
    plt.bar(range(len(combination_counts_balanced)), list(combination_counts_balanced.values()))
    plt.title('Class Combinations After Undersampling')
    plt.xlabel('Class Combinations')
    plt.ylabel('Count (log scale)')
    plt.yscale('log') #symlog

    plt.tight_layout()
    plt.show()
    
    # Print the top 15 most common classes and their counts after undersampling
    most_common_classes_after = combination_counts_balanced.most_common(15)
    print("\nTop 15 Most Common Classes After Undersampling:")
    for combination, count in most_common_classes_after:
        print(f"Class Combination: {combination}, Count: {count}")

    return df_balanced

#df = df.head(10000)
#train = undersample_class_combinations(train, label_columns)
#train

In [2]:
import json

with open("/kaggle/input/most-important-words/most_important_words.json", 'r') as json_file:
    loaded_list = json.load(json_file)

loaded_list[0]

['humiliation',
 'stepfather',
 'raped',
 'underage',
 'autistic',
 'eat',
 'transgender',
 'retard',
 'money',
 'bitch',
 'racist',
 'belly',
 'gay',
 'asexual',
 'dysphoria',
 'torture',
 'grief',
 'throat',
 'kidnapping',
 'abduction',
 'blood',
 'surgery',
 'cutting',
 'suicide',
 'omega',
 'incest',
 'contraction',
 'abortion',
 'miscarriage',
 'belly',
 'lan',
 'king']

In [3]:
from transformers import BertTokenizer, BertModel, AutoTokenizer

MAX_LEN = 510
#MAX_LEN = 4096

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#tokenizer = AutoTokenizer.from_pretrained('google/bigbird-roberta-base', max_length=MAX_LEN)

for word in loaded_list[0]:
    print(word, tokenizer.tokenize(word))

humiliation ['▁humiliation']
stepfather ['▁step', 'father']
raped ['▁raped']
underage ['▁underage']
autistic ['▁autistic']
eat ['▁eat']
transgender ['▁transgender']
retard ['▁retard']
money ['▁money']
bitch ['▁bitch']
racist ['▁racist']
belly ['▁belly']
gay ['▁gay']
asexual ['▁a', 'sexual']
dysphoria ['▁dysph', 'oria']
torture ['▁torture']
grief ['▁grief']
throat ['▁throat']
kidnapping ['▁kidnapping']
abduction ['▁abduction']
blood ['▁blood']
surgery ['▁surgery']
cutting ['▁cutting']
suicide ['▁suicide']
omega ['▁omega']
incest ['▁incest']
contraction ['▁contraction']
abortion ['▁abortion']
miscarriage ['▁miscarriage']
belly ['▁belly']
lan ['▁lan']
king ['▁king']


In [4]:
tokenizer.tokenize("general-purpose architectures bert gpt")

['▁general', '-', 'purpose', '▁architectures', '▁', 'bert', '▁g', 'pt']

In [5]:
tokenized_list = []

for sublist in loaded_list:
    tokenized_sublist = []
    for word in sublist:
        tokenzied_word = tokenizer.tokenize(word)
        tokenized_sublist.append(tokenzied_word)
    tokenized_sublist = [item for sublist in tokenized_sublist for item in sublist] # flatten
    tokenized_list.append(tokenized_sublist)

tokenized_list[0:1]

[['▁humiliation',
  '▁step',
  'father',
  '▁raped',
  '▁underage',
  '▁autistic',
  '▁eat',
  '▁transgender',
  '▁retard',
  '▁money',
  '▁bitch',
  '▁racist',
  '▁belly',
  '▁gay',
  '▁a',
  'sexual',
  '▁dysph',
  'oria',
  '▁torture',
  '▁grief',
  '▁throat',
  '▁kidnapping',
  '▁abduction',
  '▁blood',
  '▁surgery',
  '▁cutting',
  '▁suicide',
  '▁omega',
  '▁incest',
  '▁contraction',
  '▁abortion',
  '▁miscarriage',
  '▁belly',
  '▁lan',
  '▁king']]

In [ ]:
from sklearn.feature_extraction import text
from bs4 import BeautifulSoup
import string
import re
from tqdm.notebook import tqdm
import gzip
from tqdm import tqdm
tqdm.pandas()

END_TOKENS_LEN = 100
TOKEN_NEIGHBOURS = 8 #4

def preprocess_function(example):
    #Remove irrelevant data and special character:
    example["text"] = BeautifulSoup(example["text"], "html.parser").get_text(strip=True) #remove links
    example["text"] = re.sub(r'\xa0', ' ', str(example["text"])) # remove further html stuff
    example["text"] = re.sub(r'\n', ' ', str(example["text"])) # remove paragraphs
    example["text"] = re.sub(r'\s+', ' ', str(example["text"])) # multiple spaces to one space
    example["text"] = re.sub(r'[^\x00-\x7F]+', '', str(example["text"])) #remove non asci
        
    return example

def pad(text, max_sequence_length, token):
    num_padding = max_sequence_length - len(text)

    # Create a padding mask
    padding_mask = np.zeros(max_sequence_length, dtype=int)

    # Pad the list with zeros to the maximum length
    padded_list = np.pad(text, (0, num_padding), mode='constant', constant_values=token)

    # Update the padding mask to indicate the padded positions
    padding_mask[:len(text)] = 1
      
    return padded_list, padding_mask

def extract_subtext(data):
    data = preprocess_function(data)
    
    input_tokens = tokenizer.tokenize(data["text"])
    
    final_text = []
    token_count = 0
    
    if len(input_tokens) > MAX_LEN:
        end_of_text = input_tokens[-END_TOKENS_LEN:]
        del input_tokens[-END_TOKENS_LEN:]
        
        for ranking, important_words in enumerate(tokenized_list):
            for i, word in enumerate(input_tokens):
                if word in important_words:
                    start_index = max(0, i - TOKEN_NEIGHBOURS)
                    end_index = min(len(input_tokens), i + (TOKEN_NEIGHBOURS + 1))

                    chosen_tokens = input_tokens[start_index:end_index]
                    final_text.extend(chosen_tokens)

                    del input_tokens[start_index:end_index]

                    token_count += len(chosen_tokens)

                if token_count >= MAX_LEN - END_TOKENS_LEN:
                    break
            if token_count >= MAX_LEN - END_TOKENS_LEN:
                break
    
        if len(final_text) > MAX_LEN - END_TOKENS_LEN:
            final_text = final_text[:MAX_LEN - END_TOKENS_LEN]

        token_count = token_count + len(end_of_text)
        final_text = final_text + end_of_text
        
        final_text = final_text[:MAX_LEN] 
    else:
        final_text = input_tokens
        
    final_text[0] = tokenizer.cls_token
    final_text[-1] = tokenizer.sep_token
    
    final_text, mask = pad(final_text, MAX_LEN, tokenizer.pad_token)
    final_tokens = tokenizer.convert_tokens_to_ids(final_text)
    
    #data["tokens"] = final_tokens
    data["text"] = final_tokens
    data["mask"] = mask
    
    return data
    

train = train.progress_apply(lambda x: extract_subtext(x), axis=1)
val = val.progress_apply(lambda x: extract_subtext(x), axis=1)
#train.iloc[1:2].progress_apply(lambda x: extract_subtext(x), axis=1)[["mask", "text"]]
train

 12%|█▏        | 37998/307102 [39:45<7:36:16,  9.83it/s] 

In [ ]:
train_output_path = 'train_cutted_tokenized.pkl'
val_output_path = 'val_cutted_tokenized.pkl'

# Use the to_pickle method to save the DataFrame to a pickle file
train.to_pickle(train_output_path)
val.to_pickle(val_output_path)

# Confirm that the file has been saved
print(f"DataFrame saved to {train_output_path} and {val_output_path}")